In [104]:
#必要なライブラリをインポート
import importlib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score

In [96]:
#最初の設定
y='SalePrice'
main_feature_num=10
filenames=['train.csv','test.csv']
index=["linearregression","randomforestregressor","elasticnet"]
model_series=pd.Series([True,True,True],index=index)

def make_modellist(model_series):
    modelset=[]
    if model_series['linearregression']:
        linearregression=LinearRegression()
        modelset.append(linearregression)
    if model_series['randomforestregressor']:
        randomforestregressor=RandomForestRegressor()
        modelset.append(randomforestregressor)
    if model_series['elasticnet']:
        elasticnet=ElasticNet()
        modelset.append(elasticnet)
    return modelset

In [91]:
#自作のモジュールをインポート
import data_importer
import data_handler
import data_analyzer

In [92]:
#モジュールの更新
importlib.reload(data_importer)
importlib.reload(data_handler)
importlib.reload(data_analyzer)

<module 'data_analyzer' from '/Users/n.ishida/Documents/training/gittraining/py_autoML/data_analyzer.py'>

In [93]:
#訓練用データをインポート
[train_data,test_data]=data_importer.data_import(filenames,pd)

In [94]:
#データの基礎情報表示
data_shape=data_handler.elemental_info(train_data)
#nullが2割以上あるカラムを一旦消去
train_data=data_handler.null_handling(train_data,0.8)
#データの種類と数を確認
data_handler.print_type_of_data(train_data,pd)
#オブジェクト型のカラムを消去
train_data=data_handler.object_number_of_unique(train_data)
#回帰用の最後のミニ調整
train_data=data_handler.mini_handling(train_data)
#解析用に説明変数と目的変数を分ける
[train_dataX,train_datay]=data_handler.make_data_for_analysis(train_data,y)

データを表示

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2

In [97]:
#ランダムフォレストで重要度上位に絞ったデータセット
main_data=data_analyzer.analysis_for_importance(train_dataX,train_datay,
                                                RandomForestRegressor,pd,main_feature_num)
#複数モデルが入ったセットを作る
model_set=make_modellist(model_series)

              importances
OverallQual      0.638774
GrLivArea        0.092993
BsmtFinSF1       0.033828
TotalBsmtSF      0.033496
2ndFlrSF         0.030742
1stFlrSF         0.023825
YearRemodAdd     0.014086
GarageCars       0.012802
YearBuilt        0.012220
LotArea          0.009687


/Users/n.ishida/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [117]:
def model_score(model_set,dataX,datay):
    print(cross_val_score(model_set[0],dataX,datay,cv=5).mean())
    
    

In [118]:
model_score(model_set,train_dataX,train_datay)

0.7679352325870177


In [112]:
help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv='warn', n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score='raise-deprecating')
    Evaluate a score by cross-validation
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like
        The data to fit. Can be for example a list, or an array.
    
    y : array-like, optional, default: None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like, with shape (n_samples,), optional
        Group labels for the samples used while splitting the dataset into
        train/test set.
    
    scoring : string, callable or None, optional, default: None
        A string (see model evaluation documentat

In [113]:
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble.forest:

class RandomForestRegressor(ForestRegressor)
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |      .. versionchanged:: 0.20
 |         The default value of ``n_estimators`` will change from 10 in
 |         version 0.20 to 100 in version 0.22.
 |  
 |  criterion : string, optional (default="mse")
 |      The function to measure the quality of a split. Supported criteria
 |  

In [114]:
help(LinearRegression)

Help on class LinearRegression in module sklearn.linear_model.base:

class LinearRegression(LinearModel, sklearn.base.RegressorMixin)
 |  Ordinary least squares Linear Regression.
 |  
 |  Parameters
 |  ----------
 |  fit_intercept : boolean, optional, default True
 |      whether to calculate the intercept for this model. If set
 |      to False, no intercept will be used in calculations
 |      (e.g. data is expected to be already centered).
 |  
 |  normalize : boolean, optional, default False
 |      This parameter is ignored when ``fit_intercept`` is set to False.
 |      If True, the regressors X will be normalized before regression by
 |      subtracting the mean and dividing by the l2-norm.
 |      If you wish to standardize, please use
 |      :class:`sklearn.preprocessing.StandardScaler` before calling ``fit`` on
 |      an estimator with ``normalize=False``.
 |  
 |  copy_X : boolean, optional, default True
 |      If True, X will be copied; else, it may be overwritten.
 | 